In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(216716096, 6)


,session,aid,ts,type,hour,day
0,0,1517085,1659304800,0,79200,338400
1,0,1563459,1659304904,0,79304,338504
2,0,1309446,1659367439,0,55439,401039
3,0,16246,1659367719,0,55719,401319
4,0,1781822,1659367871,0,55871,401471


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
0,0.000000,44,36,51536.363636,397136.363636
1,0.029412,34,30,49709.470588,253003.588235
2,0.000000,17,16,43367.058824,297484.705882
3,0.102464,2557,1277,50332.984748,293482.926085
4,0.042254,213,138,54283.009390,294012.586854


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
1751748,0.000000,9756.791992,184546.390625
1015389,0.306141,17433.667969,184165.546875
1470784,0.453253,18976.771484,178647.906250
275417,0.462910,4457.587891,119131.195312
792125,0.617213,16551.763672,125067.531250


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,44,36,51536.363281,397136.37500,1.222222,0.000000,19491.478516,150098.031250
1,0.029412,34,30,49709.468750,253003.59375,1.133333,0.171499,18507.193359,154289.843750
2,0.000000,17,16,43367.058594,297484.71875,1.062500,0.000000,10848.244141,130795.054688
3,0.102464,2557,1277,50332.984375,293482.93750,2.002349,0.348905,16573.421875,148353.406250
4,0.042254,213,138,54283.007812,294012.59375,1.543478,0.201641,17723.203125,168626.953125


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
item_features.to_parquet('../../data/item_user_features/item4_LB.pqt')